In [ ]:
class ThreeInputsNet(nn.Module):
  def __init__(self, n_tokens, n_cat_features, hid_size = 64, n_max = 2):
    super(self.__class__, self).__init__()
    self.emb1 = nn.Embedding(
        num_embeddings = n_tokens,
        embedding_dim = hid_size
    )
    self.reorder1 = Reorder()
    self.conv1_1 = nn.Conv1d(
        in_channels = hid_size,
        out_channels = hid_size,
        kernel_size = 3
    )
    self.relu1_1 = nn.ReLU()
    self.conv1_2 = nn.Conv1d(
        in_channels=hid_size,
        out_channels=hid_size,
        kernel_size=4
    )
    self.bn1 = nn.BatchNorm1d(hid_size)
    self.relu1_2 = nn.ReLU()
    self.adaptive_pool1 = nn.AdaptiveMaxPool1d(output_size=n_max)
    self.flatten1 = nn.Flatten()
    # self.linear1 = nn.Linear(hid_size * n_max, 1)

    self.emb2 = nn.Embedding(
        num_embeddings = n_tokens,
        embedding_dim=hid_size,
    )
    self.reorder2 = Reorder()
    self.conv2_1 = nn.Conv1d(
        in_channels=hid_size,
        out_channels=hid_size,
        kernel_size=3
    )
    self.relu2_1 = nn.ReLU()
    self.conv2_2 = nn.Conv1d(
        in_channels=hid_size,
        out_channels=hid_size,
        kernel_size=4
    )
    self.bn2 = nn.BatchNorm1d(hid_size)
    self.relu2_2 = nn.ReLU()
    self.adaptive_pool2 = nn.AdaptiveMaxPool1d(output_size=n_max)
    self.flatten2 = nn.Flatten()
    # self.linear2 = nn.Linear(hid_size * n_max, 1)

    self.linear3 = nn.Linear(n_cat_features, hid_size * n_max)

    self.linear123 = nn.Linear(hid_size * n_max + hid_size * n_max + hid_size * n_max, 1)

  def forward(self, x):
    full_descr, title, cat_features = x

    full_descr = self.emb1(full_descr)
    full_descr = self.reorder1(full_descr)
    full_descr = self.conv1_1(full_descr)
    full_descr = self.relu1_1(full_descr)
    full_descr = self.conv1_2(full_descr)
    full_descr = self.bn1(full_descr)
    full_descr = self.relu1_2(full_descr)
    full_descr = self.adaptive_pool1(full_descr)
    full_descr = self.flatten1(full_descr)
    # full_descr = self.linear1(full_descr)

    title = self.emb1(title)
    title = self.reorder1(title)
    title = self.conv1_1(title)
    title = self.relu1_1(title)
    title = self.conv1_2(title)
    title = self.bn1(title)
    title = self.relu1_2(title)
    title = self.adaptive_pool1(title)
    title = self.flatten1(title)
    # title = self.linear1(title)

    cat_features = self.linear3(cat_features)

    x = torch.cat((full_descr, title, cat_features), dim=1)
    x = self.linear123(x)

    return x